<a href="https://colab.research.google.com/github/tkdlqh2/wanted_8/blob/main/wanted_8/4_NLG_%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD/na_hyun/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! git clone https://github.com/seujung/KoBART-summarization.git

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 142 (delta 77), reused 84 (delta 31), pack-reused 0
Receiving objects: 100% (142/142), 37.23 MiB | 25.03 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [2]:
cd /content/KoBART-summarization

/content/KoBART-summarization


In [3]:
! pip install -r requirements.txt
'''pandas
torch==1.10.0
transformers==4.16.2
pytorch-lightning==1.5.10
streamlit==1.2.0'''

     |████████████████████████████████| 3.5 MB 7.9 MB/s 
     |████████████████████████████████| 527 kB 52.1 MB/s 
     |████████████████████████████████| 9.1 MB 52.6 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 64.3 MB/s 
     |████████████████████████████████| 6.5 MB 49.7 MB/s 
     |████████████████████████████████| 596 kB 66.3 MB/s 
     |████████████████████████████████| 134 kB 75.5 MB/s 
     |████████████████████████████████| 397 kB 77.4 MB/s 
     |████████████████████████████████| 952 kB 59.4 MB/s 
     |████████████████████████████████| 829 kB 61.7 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
     |████████████████████████████████| 111 kB 77.5 MB/s 
     |████████████████████████████████| 164 kB 68.5 MB/s 
     |████████████████████████████████| 4.3 MB 37.6 MB/s 
     |████████████████████████████████| 181 kB 74.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |█████████████

'pandas\ntorch==1.10.0\ntransformers==4.16.2\npytorch-lightning==1.5.10\nstreamlit==1.2.0'

In [6]:
# ! pip install konlpy

In [7]:
from train import *
from dataset import *
from rouge_metric import Rouge

In [8]:
import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

In [9]:
parser = argparse.ArgumentParser(description='KoBART Summarization')

parser.add_argument('--checkpoint_path', default='/content/drive/MyDrive/AI/logs/model_chp/epoch=02-val_loss=2.733.ckpt',           # 모델 체크포인트 경로 입력
                    type=str,
                    help='checkpoint path')

parser.add_argument('--hparams_file', default = '/content/drive/MyDrive/AI/logs/tb_logs/default/version_0/hparams.yaml',              # 하이퍼파라미터 정보가 담긴 yaml 파일 경로 입력
                    type=str, help='input hparams file path')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [10]:
class Base(pl.LightningModule):
    def __init__(self, hparams, **kwargs) -> None:
        super(Base, self).__init__()
        self.hparams = hparams

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)

        parser.add_argument('--batch-size',
                            type=int,
                            default=14,
                            help='batch size for training (default: 96)')

        parser.add_argument('--lr',
                            type=float,
                            default=3e-5,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')

        parser.add_argument('--model_path',
                            type=str,
                            default=None,
                            help='kobart model path')
        return parser

    def configure_optimizers(self):
        # Prepare optimizer
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.lr, correct_bias=False)

        num_workers = self.hparams.num_workers
        data_len = len(self.train_dataloader().dataset)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]

In [11]:
class KoBARTConditionalGeneration(Base):
    def __init__(self, hparams, trainer=None, **kwargs):
        super(KoBARTConditionalGeneration, self).__init__(hparams, trainer, **kwargs)
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):

        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()
        
        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          decoder_input_ids=inputs['decoder_input_ids'],
                          decoder_attention_mask=decoder_attention_mask,
                          labels=inputs['labels'], return_dict=True)


    def training_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs.loss
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs['loss']
        return (loss)

    def validation_epoch_end(self, outputs):
        losses = []
        for loss in outputs:
            losses.append(loss)
        self.log('val_loss', torch.stack(losses).mean(), prog_bar=True)

    def test_step(self, batch, batch_idx):
        score = {'rouge-1':{'r':0, 'p':0, 'f':0}, 
                 'rouge-2':{'r':0, 'p':0, 'f':0}, 
                 'rouge-l':{'r':0, 'p':0, 'f':0}}
        rouge = Rouge()
        
        x = batch['input_ids']
        y = batch['label']

        output = self.model.generate(x, eos_token_id=1, max_length=512, num_beams=5)
        
        
        for i in range(len(output)):

            predict = self.tokenizer.decode(output[i], skip_special_tokens=True)
            label = y[i]
            
            s = rouge.get_scores(predict, label)[0]
            
            
            score['rouge-1']['r'] += s['rouge-1']['r']
            score['rouge-1']['p'] += s['rouge-1']['p']
            score['rouge-1']['f'] += s['rouge-1']['f']
            
            score['rouge-2']['r'] += s['rouge-2']['r']
            score['rouge-2']['p'] += s['rouge-2']['p']
            score['rouge-2']['f'] += s['rouge-2']['f']
            
            score['rouge-l']['r'] += s['rouge-l']['r']
            score['rouge-l']['p'] += s['rouge-l']['p']
            score['rouge-l']['f'] += s['rouge-l']['f']
            
            
        score['rouge-1']['r'] /= len(output)
        score['rouge-1']['p'] /= len(output)
        score['rouge-1']['f'] /= len(output)
            
        score['rouge-2']['r'] /= len(output)
        score['rouge-2']['p'] /= len(output)
        score['rouge-2']['f'] /= len(output)
            
        score['rouge-l']['r'] /= len(output)
        score['rouge-l']['p'] /= len(output)
        score['rouge-l']['f'] /= len(output)
        
        return (score)
    
    
    
    def test_epoch_end(self, outputs):
        score = {'rouge-1':{'r':0, 'p':0, 'f':0}, 'rouge-2':{'r':0, 'p':0, 'f':0}, 'rouge-l':{'r':0, 'p':0, 'f':0}}
        
        for s in outputs:
            score['rouge-1']['r'] += s['rouge-1']['r']
            score['rouge-1']['p'] += s['rouge-1']['p']
            score['rouge-1']['f'] += s['rouge-1']['f']
            
            score['rouge-2']['r'] += s['rouge-2']['r']
            score['rouge-2']['p'] += s['rouge-2']['p']
            score['rouge-2']['f'] += s['rouge-2']['f']
            
            score['rouge-l']['r'] += s['rouge-l']['r']
            score['rouge-l']['p'] += s['rouge-l']['p']
            score['rouge-l']['f'] += s['rouge-l']['f']
            
        
        score['rouge-1']['r'] /= len(outputs)
        score['rouge-1']['p'] /= len(outputs)
        score['rouge-1']['f'] /= len(outputs)
            
        score['rouge-2']['r'] /= len(outputs)
        score['rouge-2']['p'] /= len(outputs)
        score['rouge-2']['f'] /= len(outputs)
            
        score['rouge-l']['r'] /= len(outputs)
        score['rouge-l']['p'] /= len(outputs)
        score['rouge-l']['f'] /= len(outputs)
        
        
        df = pd.DataFrame(score)
        df.to_csv(os.path.join(args.default_root_dir, 'rouge_score.csv'))
        
        print(df)

In [13]:
if __name__ == '__main__':
    parser = Base.add_model_specific_args(parser)
    parser = ArgsBase.add_model_specific_args(parser)
    parser = KobartSummaryModule.add_model_specific_args(parser)
    parser = pl.Trainer.add_argparse_args(parser)
    tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
    args = parser.parse_args()
    logging.info(args)

    dm = KobartSummaryModule(args.train_file,
                        args.test_file,
                        tokenizer,
                        batch_size=args.batch_size,
                        max_len=args.max_len,
                        num_workers=args.num_workers)
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                       dirpath=args.default_root_dir,
                                                       filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                       verbose=True,
                                                       save_last=True,
                                                       mode='min',
                                                       save_top_k=3)
    
    early_stopping = pl.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,   # metric 성능이 몇 번의 epoch가 향상 되지않을 때 학습을 멈출건지 지정
        verbose=True,
        mode='min'
    )

    tb_logger = pl_loggers.TensorBoardLogger(os.path.join(args.default_root_dir, 'tb_logs'))
    lr_logger = pl.callbacks.LearningRateMonitor()
    trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                            callbacks=[checkpoint_callback, early_stopping, lr_logger])


    with open(args.hparams_file) as f:
        hparams = yaml.load(f, Loader=yaml.Loader)
        
    model=KoBARTConditionalGeneration.load_from_checkpoint(args.checkpoint_path, hparams=hparams)
    

    trainer.test(model=model, datamodule=dm, verbose=True)

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

usage: ipykernel_launcher.py [-h] [--checkpoint_path CHECKPOINT_PATH]
                             [--hparams_file HPARAMS_FILE]
                             [--batch-size BATCH_SIZE] [--lr LR]
                             [--warmup_ratio WARMUP_RATIO]
                             [--model_path MODEL_PATH]
                             [--train_file TRAIN_FILE] [--test_file TEST_FILE]
                             [--batch_size BATCH_SIZE] [--max_len MAX_LEN]
                             [--num_workers NUM_WORKERS] [--logger [LOGGER]]
                             [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                             [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                             [--default_root_dir DEFAULT_ROOT_DIR]
                             [--gradient_clip_val GRADIENT_CLIP_VAL]
                             [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                             [--process_position PROCESS_POSITION]
                         

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  try:
